Import all the Dependencies

In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML

Set all the Constants

In [2]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS=3
EPOCHS=50

Import data into tensorflow dataset object

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "datasets",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 14000 files belonging to 14 classes.


In [4]:
class_names = dataset.class_names
class_names

['Bacterial Spot of Tomato',
 'Black Rot of Grapes',
 'Common Rust in Corn',
 'Corn Healthy Leaf',
 'Early Blight of Potato',
 'Early Blight of Tomato',
 'Esca (Black Measles) of Grapes',
 'Grape Healthy Leaf',
 'Late Blight of Potato',
 'Late Blight of Tomato',
 'Nothern Corn Leaf Blight',
 'Potato Healthy Leaf',
 'Tomato Healthy Leaf',
 'Tomato Yellow Leaf Curl Virus (TYLCV)']

In [5]:
for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())

(32, 256, 256, 3)
[ 2  7  2 12 10 13 13  3  5  5 13  2  4 10 11  6  7 13  3  2  3  1  8  2
 10  5 13  5  0  7  2 12]


Function to Split Dataset

1. Training: Dataset to be used while training
2. Validation: Dataset to be tested against while training
3. Test: Dataset to be tested against after we trained a model

no of images/batch size

In [6]:
len(dataset)

438

In [7]:
train_size = 0.7
len(dataset)*train_size

306.59999999999997

In [8]:
train_ds = dataset.take(306)
len(train_ds)

306

In [9]:
test_ds = dataset.skip(306)
len(test_ds)

132

In [10]:
val_size=0.15
len(dataset)*val_size

65.7

In [11]:
val_ds = test_ds.take(65)
len(val_ds)

65

In [12]:
test_ds = test_ds.skip(65)
len(test_ds)

67

In [13]:
def get_dataset_partitions_tf(ds, train_split=0.7, val_split=0.15, test_split=0.15, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [14]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [15]:
len(train_ds)

306

In [16]:
len(val_ds)

65

In [17]:
len(test_ds)

67

Cache, Shuffle and Prefetch the Dataset

In [18]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

__Creating a Layer for Resizing and Normalization__ <br>
Before feed images to network, they should be resizing to the desired size. 
Moreover, to improve model performance, it should normalize the image pixel value (keeping them in range 0 and 1 by dividing by 256).
This should happen while training as well as inference. Hence that can add as a layer in Sequential Model.

This will be useful when done with the training and start using the model for predictions. 
At that time somone can supply an image that is not (256,256) and this layer will resize it

In [19]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])

__Model Architecture__
<br>
Used a CNN coupled with a Softmax activation in the output layer. 
Initial layers for resizing and normalization.

In [20]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 14

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (32, 256, 256, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (32, 254, 254, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (32, 127, 127, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (32, 125, 125, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (32, 62, 62, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (32, 60, 60, 64)          36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (32, 30, 30, 64)         

Compiling the Model

`adam` Optimizer,
`SparseCategoricalCrossentropy` for losses, 
`accuracy` as a metric

In [22]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [23]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=50,
)

Epoch 1/50
306/306 [==============================] - 558s 2s/step - loss: 1.6750 - accuracy: 0.4270 - val_loss: 0.8010 - val_accuracy: 0.7091
Epoch 2/50
306/306 [==============================] - 489s 2s/step - loss: 0.6774 - accuracy: 0.7630 - val_loss: 0.6085 - val_accuracy: 0.7928
Epoch 3/50
306/306 [==============================] - 483s 2s/step - loss: 0.4541 - accuracy: 0.8404 - val_loss: 0.3456 - val_accuracy: 0.8798
Epoch 4/50
306/306 [==============================] - 484s 2s/step - loss: 0.3289 - accuracy: 0.8816 - val_loss: 0.4142 - val_accuracy: 0.8591
Epoch 5/50
306/306 [==============================] - 479s 2s/step - loss: 0.2875 - accuracy: 0.8977 - val_loss: 0.3089 - val_accuracy: 0.8966
Epoch 6/50
306/306 [==============================] - 484s 2s/step - loss: 0.2159 - accuracy: 0.9208 - val_loss: 0.1955 - val_accuracy: 0.9308
Epoch 7/50
306/306 [==============================] - 483s 2s/step - loss: 0.1979 - accuracy: 0.9295 - val_loss: 0.2550 - val_accuracy: 0.9120

In [24]:
scores = model.evaluate(test_ds)

67/67 [==============================] - 75s 356ms/step - loss: 0.0860 - accuracy: 0.9823


In [25]:
import os
model.save(f"C:/Users/ASUS/Documents/models/3")

INFO:tensorflow:Assets written to: C:/Users/ASUS/Documents/models/3\assets
